This notebook helps to populate the table from each sheet of MorpaData excel 





---


In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
#check if numpy is installed

import subprocess

try:
    subprocess.check_call(['pip', 'show', 'numpy'])
except subprocess.CalledProcessError:
    # numpy is not installed
    %pip install numpy
else:
    print('numpy is already installed.')

numpy is already installed.


In [3]:
#check if openpyxl is installed

import subprocess

try:
    subprocess.check_call(['pip', 'show', 'openpyxl'])
except subprocess.CalledProcessError:
    # openpyxl is not installed
    %pip install openpyxl
else:
    print('openpyxl is already installed.')

openpyxl is already installed.


In [4]:
#check if pandas is installed

import subprocess

try:
    subprocess.check_call(['pip', 'show', 'pandas'])
except subprocess.CalledProcessError:
    # pandas is not installed
    %pip install pandas
else:
    print('pandas is already installed.')


pandas is already installed.


In [5]:
import pandas as pd
import numpy as np
import openpyxl

# Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
# input excel file path
inputExcelFile ="MorpaData.xlsx"

# creating or loading an excel workbook
newWorkbook = openpyxl.load_workbook(inputExcelFile)

# printing all the sheetnames in an excel file using sheetnames attribute
print('The Sheet names of the given excel file: \n')

# Getting the sheetnames as a list using the sheetnames attribute
sheetNames=newWorkbook.sheetnames

# Traversing in the sheetNames list
for name in sheetNames:
   print(name)

The Sheet names of the given excel file: 

Üye Listesi
Giriş Logları
Oyun
Konu anlatımı
çalışmalar
Sınav
konu


In [7]:
# get the member data from excel

member_df = pd.read_excel('MorpaData.xlsx',sheet_name='Üye Listesi')
print(member_df.head(10),'\n\n')

print(member_df.info())

# get the unique number of members

print(member_df['Üye'].nunique())

       Üye  Şehir    İlçe       Okul  Sınıf
0  2070306  10826  162678  120166963      4
1  2910824  16378  624995  813905547      6
2  3045726  10989  721319  231052437      4
3  3661377  10185  376777  733057858      8
4  3794510  11631  709268   72006202      4
5  3969040  11631  626646  305454095      5
6  5084112  11631   23819  659915466      8
7  6789503  15012  248394  310622551      3
8  7221591  11631  485022  529623633      3
9  7899454  14951  599378  265076040      8 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Üye     1999 non-null   int64
 1   Şehir   1999 non-null   int64
 2   İlçe    1999 non-null   int64
 3   Okul    1999 non-null   int64
 4   Sınıf   1999 non-null   int64
dtypes: int64(5)
memory usage: 78.2 KB
None
1999


In [8]:
# get the Login logs data from excel

login_df = pd.read_excel('MorpaData.xlsx',sheet_name='Giriş Logları')
login_df['Giriş Zamanı'] = pd.to_datetime(login_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
login_df['Çıkış Zamanı'] = pd.to_datetime(login_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')
print(login_df.head(10),'\n\n')

print(login_df.info(),'\n\n')


# get the unique number of members on login data
print("unique member count on login data: ",login_df['Üye'].nunique())

   GirişID      Üye            Giriş Zamanı            Çıkış Zamanı  \
0   181915  2070306 2021-09-09 20:54:37.683 2021-09-09 21:46:55.070   
1   128004  2910824 2021-09-07 20:33:32.583 2021-09-07 21:25:53.753   
2   178974  2910824 2021-09-09 20:18:36.960 2021-09-09 20:58:58.370   
3   182887  2910824 2021-09-09 21:10:33.563 2021-09-09 22:04:55.467   
4   238434  2910824 2021-09-12 10:59:55.210 2021-09-12 11:04:15.233   
5   244552  2910824 2021-09-12 12:32:28.067 2021-09-12 13:28:49.353   
6   273273  2910824 2021-09-13 13:25:51.117 2021-09-13 13:58:05.470   
7   273943  2910824 2021-09-13 13:58:05.470 2021-09-13 14:36:48.393   
8   285795  2910824 2021-09-13 18:44:16.997 2021-09-13 19:44:59.743   
9   345946  2910824 2021-09-15 17:06:06.027 2021-09-15 17:51:50.893   

   Süre Dakika  
0           52  
1           52  
2           40  
3           54  
4            5  
5           56  
6           33  
7           38  
8           60  
9           45   


<class 'pandas.core.frame.Da

In [9]:
# get the lecturing logs data from excel

lecturing_df = pd.read_excel('MorpaData.xlsx',sheet_name='Konu anlatımı')
lecturing_df['Giriş Zamanı'] = pd.to_datetime(lecturing_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
lecturing_df['Çıkış Zamanı'] = pd.to_datetime(lecturing_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')

# calculate the duration of the lecturing

lecturing_df['lec_Süre'] = (lecturing_df['Çıkış Zamanı'] - lecturing_df['Giriş Zamanı']).dt.total_seconds().astype(int)

print(lecturing_df.head(10),'\n\n')

print(lecturing_df.info(),'\n\n')


# get the unique number of members on lecturing data
print("Unique number of users on lecturing: ", lecturing_df['Üye'].nunique())

       Üye  Eğitim Materyali  Eğitim Materyali Türü  Ders   Konu  \
0  2070306      518826088921                      2     2   9864   
1  2070306      608022794749                      2     2   9864   
2  2910824      633794085515                      2     2   8964   
3  2910824      787304215519                      2     9   9535   
4  2910824      137540475371                      2     3   9659   
5  2910824      799934114128                      2    11   4421   
6  2910824      139874336279                      2     1  10002   
7  2910824      884070756143                      2     2   8965   
8  2910824      884070756143                      2     2   8965   
9  2910824      139874336279                      2     1  10002   

   Katılım Oranı  Performans            Giriş Zamanı            Çıkış Zamanı  \
0             75           0 2021-09-09 20:55:06.357 2021-09-09 21:00:04.783   
1            100           0 2021-09-09 21:00:04.797 2021-09-09 21:46:55.070   
2          

In [10]:
# get the studies data from excel

studies_df = pd.read_excel('MorpaData.xlsx',sheet_name='çalışmalar')
studies_df['Giriş Zamanı'] = pd.to_datetime(studies_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
studies_df['Çıkış Zamanı'] = pd.to_datetime(studies_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')

studies_df['stud_Süre'] = (studies_df['Çıkış Zamanı'] - studies_df['Giriş Zamanı']).dt.total_seconds().astype(int)

print(studies_df.head(10),'\n\n')

print(studies_df.info(),'\n\n')

# get the unique number of members on studies data
print("unique member count on studies data: ",studies_df['Üye'].nunique())

       Üye  Eğitim Materyali  Eğitim Materyali Türü  Ders   Konu  \
0  2910824      291765069973                      3     2   8964   
1  2910824      383712317425                      3     9   9535   
2  2910824      291765069973                      3     2   8964   
3  2910824      450320146095                      3     3   9659   
4  2910824      460263704270                      3    11   4421   
5  2910824      170232354529                      3     2   8965   
6  2910824      343498413848                      3     1  10002   
7  2910824      924340585642                      3     2   8966   
8  2910824      990590760357                      3     3   9661   
9  2910824       90390416811                      3     2   8971   

   Katılım Oranı  Performans            Giriş Zamanı            Çıkış Zamanı  \
0            100          62 2021-09-07 20:34:09.977 2021-09-07 20:37:27.890   
1            100          90 2021-09-09 21:18:02.070 2021-09-09 22:04:55.470   
2          

In [11]:
# get the exams data from excel

exams_df = pd.read_excel('MorpaData.xlsx',sheet_name='Sınav')
exams_df['Giriş Zamanı'] = exams_df['Giriş Zamanı'].apply(lambda x: x.strftime("%d/%m/%Y  %H:%M:%S"))
exams_df['Çıkış Zamanı'] = exams_df['Çıkış Zamanı'].apply(lambda x: x.strftime("%d/%m/%Y  %H:%M:%S"))

exams_df['Giriş Zamanı'] = pd.to_datetime(exams_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
exams_df['Çıkış Zamanı'] = pd.to_datetime(exams_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')

exams_df['Giriş Zamanı']

print(exams_df.head(10),'\n\n')

print(exams_df.info(),'\n\n')

# get the unique number of members on exams data
print("unique member count on exams data: ",exams_df['Üye'].nunique())


# wrong answers are not deducted from the total score.

       Üye         Sınav   Konu  Sınav Türü  Doğru Sayısı  Yanlış Sayısı  \
0  2910824  268222943622   8964           1             8              1   
1  2910824  501151045356   9535           1             9              0   
2  2910824  934072969211   9659           1             8              2   
3  2910824  934072969211   9659           1             9              1   
4  2910824     438484723   4421           1             4              6   
5  2910824  523324425042   8965           1             0              8   
6  2910824  182157124583  10002           1             0              0   
7  2910824  182157124583  10002           1             5              5   
8  2910824  636098373206   8971           1             9              0   
9  2910824  808509304221   9662           1             3              6   

   Boş Sayısı  Puan        Giriş Zamanı        Çıkış Zamanı  Süre  GirişLog  
0           1    80 2021-09-07 20:40:54 2021-09-07 20:45:06   237    128004  
1      

In [12]:
# get the subject data from excel

subject_df = pd.read_excel('MorpaData.xlsx',sheet_name='konu')
subject_df.head(10)

#sort according to aktif materyal sayısı

subject_df.sort_values(by=['Aktif Materyal Sayısı'],ascending=False)


,Sınıf,Ders,Konu,Aktif Materyal Sayısı,Toplam Materyal Sayısı
4499,25,2,899,58,58
4110,22,1,7699,42,49
5324,36,3,6749,39,39
3916,20,3,2192,38,39
4728,27,3,4338,37,37
...,...,...,...,...,...
4794,28,2,4585,0,1
4795,28,2,4586,0,1
4796,28,2,4587,0,1
872,4,2,9870,0,3


In [13]:
#left join the exam and subject dataframes on Konu column

exam_subject_df = pd.merge(exams_df,subject_df,how='left',left_on='Konu',right_on='Konu')

# change column names for further join operations. Sınav related columns are renamed with S_ prefix
exam_subject_df.rename(
    columns={"Giriş Zamanı": "S_Giriş Zamanı", "Çıkış Zamanı": "S_Çıkış Zamanı", "Süre":"S_süre"},
    inplace=True,
)


In [14]:
# drop non-numeric sınıf rows

exam_subject_df = exam_subject_df[exam_subject_df['Sınıf'].notna()]

exam_subject_df


# check data type of exam subject dataframe

exam_subject_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7878 entries, 0 to 8312
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Üye                     7878 non-null   int64         
 1   Sınav                   7878 non-null   int64         
 2   Konu                    7878 non-null   int64         
 3   Sınav Türü              7878 non-null   int64         
 4   Doğru Sayısı            7878 non-null   int64         
 5   Yanlış Sayısı           7878 non-null   int64         
 6   Boş Sayısı              7878 non-null   int64         
 7   Puan                    7878 non-null   int64         
 8   S_Giriş Zamanı          7878 non-null   datetime64[ns]
 9   S_Çıkış Zamanı          7878 non-null   datetime64[ns]
 10  S_süre                  7878 non-null   int64         
 11  GirişLog                7878 non-null   int64         
 12  Sınıf                   7878 non-null   float64 

In [15]:
# change data type to int which are not int

exam_subject_df['Sınıf'] = exam_subject_df['Sınıf'].astype('int64')
exam_subject_df['Ders'] = exam_subject_df['Ders'].astype('int64')
exam_subject_df['Aktif Materyal Sayısı'] = exam_subject_df['Aktif Materyal Sayısı'].astype('int64')
exam_subject_df['Toplam Materyal Sayısı'] = exam_subject_df['Toplam Materyal Sayısı'].astype('int64')

exam_subject_df.head(10)



,Üye,Sınav,Konu,Sınav Türü,Doğru Sayısı,Yanlış Sayısı,Boş Sayısı,Puan,S_Giriş Zamanı,S_Çıkış Zamanı,S_süre,GirişLog,Sınıf,Ders,Aktif Materyal Sayısı,Toplam Materyal Sayısı
0,2910824,268222943622,8964,1,8,1,1,80,2021-09-07 20:40:54,2021-09-07 20:45:06,237,128004,6,2,6,6
1,2910824,501151045356,9535,1,9,0,1,90,2021-09-09 21:20:08,2021-09-09 21:24:25,241,182887,6,9,6,6
2,2910824,934072969211,9659,1,8,2,0,80,2021-09-12 11:03:55,2021-09-12 12:48:24,149,238434,6,3,6,6
3,2910824,934072969211,9659,1,9,1,0,90,2021-09-13 14:03:34,2021-09-13 14:14:50,659,273943,6,3,6,6
4,2910824,438484723,4421,1,4,6,0,40,2021-09-13 18:53:41,2021-10-07 19:32:12,115,285795,6,11,5,7
5,2910824,523324425042,8965,1,0,8,2,0,2021-09-15 18:09:04,2021-09-15 18:17:35,492,349321,6,2,6,6
6,2910824,182157124583,10002,1,0,0,10,0,2021-09-17 18:27:05,2021-09-17 18:27:30,5,414086,6,1,3,3
7,2910824,182157124583,10002,1,5,5,0,50,2021-09-17 18:27:23,2021-09-17 18:28:56,73,414086,6,1,3,3
8,2910824,636098373206,8971,1,9,0,1,90,2021-09-22 20:45:43,2021-09-28 19:29:37,588,598986,6,2,6,6
9,2910824,808509304221,9662,1,3,6,1,30,2021-09-23 20:31:26,2021-09-23 20:34:46,176,639881,6,3,7,7


In [16]:
# add cumulative lecture time to exam_subject dataframe for each member according to the subject prior to exam date s_Giriş Zamanı

# calculate cumulative lecture time before the exam date for each member of each subject
exam_subject_df['cum_lecture_time'] = 0
for index, row in exam_subject_df.iterrows():
    df = lecturing_df[lecturing_df['Üye'] == row['Üye']] #Filters the lecturing_df DataFrame to select rows where the 'Üye' column matches the current row's 'Üye' value in exam_subject_df
    df = df[df['Konu'] == row['Konu']] #Further filters the df DataFrame to select rows where the 'Konu' column matches the current row's 'Konu' value in exam_subject_df
    df = df[df['Giriş Zamanı'] < row['S_Giriş Zamanı']] #Further filters the df DataFrame to select rows where the 'Giriş Zamanı' column is less than the current row's 'S_Giriş Zamanı' value in exam_subject_df. This ensures that only lecture entries that occurred before the exam start time are considered.
    df['lec_Süre'] = df['lec_Süre'].astype('int64')
    exam_subject_df.at[index,'cum_lecture_time'] = df['lec_Süre'].sum()

# calculate cumulative study time before the exam date for each member of each subject
exam_subject_df['cum_study_time'] = 0
for index, row in exam_subject_df.iterrows():
    df = studies_df[studies_df['Üye'] == row['Üye']]
    df = df[df['Konu'] == row['Konu']]
    df = df[df['Giriş Zamanı'] < row['S_Giriş Zamanı']]
    df['stud_Süre'] = df['stud_Süre'].astype('int64')
    exam_subject_df.at[index,'cum_study_time'] = df['stud_Süre'].sum()

exam_subject_df['total_study_time'] = exam_subject_df['cum_lecture_time'] + exam_subject_df['cum_study_time']
        
exam_subject_studies_df = exam_subject_df

In [17]:
# to be properly label, do necessary sorting operations on exam_subject_studies_df

exam_subject_studies_df.sort_values(by=['Üye','Sınav','Konu']).head(20)


,Üye,Sınav,Konu,Sınav Türü,Doğru Sayısı,Yanlış Sayısı,Boş Sayısı,Puan,S_Giriş Zamanı,S_Çıkış Zamanı,S_süre,GirişLog,Sınıf,Ders,Aktif Materyal Sayısı,Toplam Materyal Sayısı,cum_lecture_time,cum_study_time,total_study_time
4,2910824,438484723,4421,1,4,6,0,40,2021-09-13 18:53:41,2021-10-07 19:32:12,115,285795,6,11,5,7,438,3134,3572
6,2910824,182157124583,10002,1,0,0,10,0,2021-09-17 18:27:05,2021-09-17 18:27:30,5,414086,6,1,3,3,268,2699,2967
7,2910824,182157124583,10002,1,5,5,0,50,2021-09-17 18:27:23,2021-09-17 18:28:56,73,414086,6,1,3,3,268,2699,2967
0,2910824,268222943622,8964,1,8,1,1,80,2021-09-07 20:40:54,2021-09-07 20:45:06,237,128004,6,2,6,6,2905,197,3102
13,2910824,360065274323,8970,1,1,1,8,10,2021-09-28 19:08:42,2021-09-28 19:11:42,155,811345,6,2,6,6,443,200,643
14,2910824,360065274323,8970,1,7,2,1,70,2021-09-28 19:11:46,2021-09-28 19:18:30,376,811345,6,2,6,6,443,200,643
1,2910824,501151045356,9535,1,9,0,1,90,2021-09-09 21:20:08,2021-09-09 21:24:25,241,182887,6,9,6,6,432,2813,3245
5,2910824,523324425042,8965,1,0,8,2,0,2021-09-15 18:09:04,2021-09-15 18:17:35,492,349321,6,2,6,6,2779,3046,5825
8,2910824,636098373206,8971,1,9,0,1,90,2021-09-22 20:45:43,2021-09-28 19:29:37,588,598986,6,2,6,6,104,120,224
15,2910824,636098373206,8971,1,8,2,0,80,2021-09-29 18:57:07,2021-09-29 19:09:46,733,851286,6,2,6,6,104,3489,3593


In [23]:
# to tag the members who attempt gaming the system, I crate a deep copy of exams_subject_studies df_gtsLabeled.

df_gtsLabeled = exam_subject_studies_df.copy(deep=True)

# create a new column for labeling and overlapping time

df_gtsLabeled['Label'] = ""
df_gtsLabeled['Overlap (secs.)'] = ""

# update the index of df_gtsLabeled

df_gtsLabeled.reset_index(drop=True, inplace=True)

# then we write the conditions for labeling the data who "Gaming the System". 1 for gaming, 0 for not gaming.

for i in range(1,len(df_gtsLabeled)):
    if df_gtsLabeled['Üye'][i-1] == df_gtsLabeled['Üye'][i]:
        if df_gtsLabeled['Sınav'][i-1] == df_gtsLabeled['Sınav'][i]:
            if df_gtsLabeled['Konu'][i-1] == df_gtsLabeled['Konu'][i]:
                if (df_gtsLabeled['S_Çıkış Zamanı'][i-1] > df_gtsLabeled['S_Giriş Zamanı'][i]) and (df_gtsLabeled['Puan'][i-1] < df_gtsLabeled['Puan'][i]) and (df_gtsLabeled['GirişLog'][i-1] == df_gtsLabeled['GirişLog'][i]):
                    df_gtsLabeled['Label'][i] = 1
                    df_gtsLabeled['Overlap (secs.)'][i] = (df_gtsLabeled['S_Giriş Zamanı'][i] - df_gtsLabeled['S_Çıkış Zamanı'][i-1]).total_seconds() * -1  # this is the overlapping time in seconds 


# name the table as df for the sake of simplicity
df = df_gtsLabeled

df.head(20)

C:\Users\Samsung\AppData\Local\Temp\ipykernel_13196\2644890427.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gtsLabeled['Label'][i] = 1
C:\Users\Samsung\AppData\Local\Temp\ipykernel_13196\2644890427.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gtsLabeled['Overlap (secs.)'][i] = (df_gtsLabeled['S_Giriş Zamanı'][i] - df_gtsLabeled['S_Çıkış Zamanı'][i-1]).total_seconds() * -1  # this is the overlapping time in seconds


,Üye,Sınav,Konu,Sınav Türü,Doğru Sayısı,Yanlış Sayısı,Boş Sayısı,Puan,S_Giriş Zamanı,S_Çıkış Zamanı,S_süre,GirişLog,Sınıf,Ders,Aktif Materyal Sayısı,Toplam Materyal Sayısı,cum_lecture_time,cum_study_time,total_study_time,Label,Overlap (secs.)
0,2910824,268222943622,8964,1,8,1,1,80,2021-09-07 20:40:54,2021-09-07 20:45:06,237,128004,6,2,6,6,2905,197,3102,,
1,2910824,501151045356,9535,1,9,0,1,90,2021-09-09 21:20:08,2021-09-09 21:24:25,241,182887,6,9,6,6,432,2813,3245,,
2,2910824,934072969211,9659,1,8,2,0,80,2021-09-12 11:03:55,2021-09-12 12:48:24,149,238434,6,3,6,6,0,0,0,,
3,2910824,934072969211,9659,1,9,1,0,90,2021-09-13 14:03:34,2021-09-13 14:14:50,659,273943,6,3,6,6,591,2774,3365,,
4,2910824,438484723,4421,1,4,6,0,40,2021-09-13 18:53:41,2021-10-07 19:32:12,115,285795,6,11,5,7,438,3134,3572,,
5,2910824,523324425042,8965,1,0,8,2,0,2021-09-15 18:09:04,2021-09-15 18:17:35,492,349321,6,2,6,6,2779,3046,5825,,
6,2910824,182157124583,10002,1,0,0,10,0,2021-09-17 18:27:05,2021-09-17 18:27:30,5,414086,6,1,3,3,268,2699,2967,,
7,2910824,182157124583,10002,1,5,5,0,50,2021-09-17 18:27:23,2021-09-17 18:28:56,73,414086,6,1,3,3,268,2699,2967,1,7.0
8,2910824,636098373206,8971,1,9,0,1,90,2021-09-22 20:45:43,2021-09-28 19:29:37,588,598986,6,2,6,6,104,120,224,,
9,2910824,808509304221,9662,1,3,6,1,30,2021-09-23 20:31:26,2021-09-23 20:34:46,176,639881,6,3,7,7,0,988,988,,


In [19]:
# fill with zero if Label column is not 1

df.replace("", 0, inplace=True)

In [20]:
df['Label'].value_counts()

# we have 7878 rows data and 290 of them are labeled as 1. This means that 290 members attempted gaming the system.

0    7588
1     290
Name: Label, dtype: int64

In [21]:
# export df to csv file

df.to_csv('populated_df.csv', index=False)
